# &AElig;co
***Calc module***
<br/><br/>

✈️♻️

This code is part of the Master's Thesis "A Life Cycle Approach for Aircraft Ecodesign", by Giácomo Parolin.

``Please enter the following information:``

In [1]:
import os
os.chdir("C:/Users/giparoli/Documents/Projetos/AEco")

input_path = './Data/Iris_inputs.xlsx'
input_sheet = 'Iris'
output_path = './Outputs/'+ input_sheet + '_outputs'
database_path = './Data/database_Iris.xlsx'

iterations = 10000
chunks = 100  #'auto' or more than 500

In [2]:
from Tools_Iris import *
from Model_Iris import *

# LCI

The aircraft's life cycle was divided into four parts:
1. Development and Engineering
2. Manufacturing and Assembly
3. Operation
4. End-of-Life

First, we execute function to read inputs from designated `input_path` and transform it into Parameters and ParameterSet object `p`.

In [3]:
inputs = read_inputs(input_path, input_sheet)

In [4]:
p = ParameterSet(inputs, iterations, chunks)

Samples are stored in a xr.Dataset object accessible throught the `p.data` ParameterSet attribute.

Then, we run the function that calculates the functional unit for the study.

In [5]:
p = func_unit(p)

In [6]:
p.data["ha_fleet"].mean().compute()

<xarray.DataArray 'ha_fleet' ()>
array(2.39412833e+09)

In [7]:
p.data["ha_life"].mean().compute()

<xarray.DataArray 'ha_life' ()>
array(5688943.42381002)

<br></br>
Executing the UP reading and writing functions.

In [8]:
UP_dataframe = read_unit_processes(database_path)
UP = unit_process_dataset(UP_dataframe)

Building the empty LCI to start the calculation.

In [9]:
inventory = LCI(name=input_sheet, iterations=iterations, UP=UP, parameters=p)
inventory.run()

<xarray.Dataset>
Dimensions:         (Substances: 1835, i: 10000)
Coordinates:
    Units           (Substances) object 'kg' 'kg' 'kg' 'kg' ... 'kg' 'kBq' 'kBq'
  * Substances      (Substances) MultiIndex
  - name            (Substances) object '1,4-Butanediol' ... 'Zirconium-95'
  - compartment     (Substances) object 'Water' 'Air' 'Air' ... 'Water' 'Air'
  - subcompartment  (Substances) object 'surface water' ... 'non-urban air or...
  * i               (i) int32 0 1 2 3 4 5 6 ... 9994 9995 9996 9997 9998 9999
Data variables: (12/16)
    Office          (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Infrastructure  (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Capital         (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Materials       (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Factory         (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Logistics       (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    ...              ...
    Fuel            (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Recycling       (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Incineration    (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Landfill        (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Prototypes      (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
    Certification   (Substances, i) float64 dask.array<chunksize=(1835, 100), meta=np.ndarray>
Attributes:
    Name:     Iris

The LCI data can be accessed by the `LCI.data` attribute.

# LCIA

Extracting the characterization factors from the excel file located in `database_path`.

Creating an instance of the characterization factors class:

In [10]:
MP_data, EP_data = read_CF(database_path)
CF = CharactFactors(MP_data, EP_data, UP.Substances)

Adjusting the NLT category CFs.

In [11]:
CF.MP.dataset['NLT'].data = np.negative(CF.MP.dataset['NLT'].data)
CF.MP.to_array();

The impacts CFs can be accessed by the `CharactFactors.Midpoint.dataset` or `CharactFactors.Endpoint.dataset` attributes.

In [12]:
display(CF.MP.dataset)
display(CF.EP.dataset)

<xarray.Dataset>
Dimensions:         (Substances: 1835)
Coordinates:
  * Substances      (Substances) MultiIndex
  - name            (Substances) object '1,4-Butanediol' ... 'Zirconium-95'
  - compartment     (Substances) object 'Water' 'Air' 'Air' ... 'Water' 'Air'
  - subcompartment  (Substances) object 'surface water' ... 'non-urban air or...
Data variables: (12/18)
    FE              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    MRD             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    POF             (Substances) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    MET             (Substances) float64 0.0 0.0 0.000283 ... 0.0 0.0 0.0
    TA              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ULO             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ...              ...
    ALO             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    HT              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    IR              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    FD              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    WD              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    CC              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

<xarray.Dataset>
Dimensions:     (Categories: 18)
Coordinates:
  * Categories  (Categories) object 'CC' 'OD' 'TA' 'FE' ... 'WD' 'MRD' 'FD'
Data variables:
    HH          (Categories) float64 1.4e-06 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ED          (Categories) float64 7.93e-09 0.0 5.8e-09 ... 0.0 0.0 0.0
    RA          (Categories) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0715 0.165

<br></br>
**Executing the Impact Assessment.**

In [13]:
aeco = LCIA.build(inventory, CF)

## Midpoint Results

Raw midpoint results can be accessed by the `LCIA.MP` attribute. Mean midpoint results are calculated by the `LCIA.mean` method, with pathway set to "MP".

In [14]:
display(aeco.MP)

<xarray.Dataset>
Dimensions:         (Categories: 18, i: 10000)
Coordinates:
  * i               (i) int32 0 1 2 3 4 5 6 ... 9994 9995 9996 9997 9998 9999
  * Categories      (Categories) <U3 'FE' 'MRD' 'POF' 'MET' ... 'FD' 'WD' 'CC'
Data variables: (12/16)
    Office          (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Infrastructure  (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Capital         (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Materials       (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Factory         (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Logistics       (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    ...              ...
    Fuel            (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Recycling       (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Incineration    (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Landfill        (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Prototypes      (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
    Certification   (i, Categories) float64 dask.array<chunksize=(100, 18), meta=np.ndarray>
Attributes:
    Name:     Iris

## Endpoint Results

Raw endpoint results can be accessed by the `LCIA.EP` attribute. Mean endpoint results are calculated by the `LCIA.mean` method, with pathway set to "EP".

In [17]:
display(aeco.EP)

<xarray.Dataset>
Dimensions:         (AOP: 3, i: 10000)
Coordinates:
  * i               (i) int32 0 1 2 3 4 5 6 ... 9994 9995 9996 9997 9998 9999
  * AOP             (AOP) <U2 'HH' 'ED' 'RA'
Data variables: (12/16)
    Office          (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Infrastructure  (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Capital         (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Materials       (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Factory         (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Logistics       (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    ...              ...
    Fuel            (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Recycling       (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Incineration    (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Landfill        (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Prototypes      (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
    Certification   (i, AOP) float64 dask.array<chunksize=(100, 3), meta=np.ndarray>
Attributes:
    Name:     Iris

In [24]:
aeco.mean("EP", by="sum").compute()

<xarray.DataArray 'stack-612c3cd7be455dca013744b10c410262' (AOP: 3)>
array([2.83707451e-04, 4.83571325e-07, 4.74229246e+00])
Coordinates:
  * AOP      (AOP) <U2 'HH' 'ED' 'RA'

## Contribution to Variance (CTV)

Calculate CTV results for all inputs with midpoint and endpoint results. CTV results can be acessed by the `LCIA.CTV` attribute.

In [18]:
aeco.build_CTV(parameterset=p)

<xarray.Dataset>
Dimensions:     (Parameters: 79)
Coordinates:
  * Parameters  (Parameters) <U18 'takt' 'OEW' ... 'p_recycl_CFRP'
Data variables: (12/21)
    FE          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    MRD         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    POF         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    MET         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    TA          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    ULO         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    ...          ...
    FD          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    WD          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    CC          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    HH          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    ED          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    RA          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>

In [20]:
aeco.CTV["CC"].sortby(aeco.CTV["CC"], ascending=False).compute()

c:\Users\giparoli\Documents\Projetos\AEco\Scripts\Tools_Iris.py:307: RuntimeWarning: invalid value encountered in true_divide
  return covariance_gufunc(x, y) / (x.std(axis=-1) * y.std(axis=-1))
c:\Users\giparoli\Documents\Projetos\AEco\Scripts\Tools_Iris.py:307: RuntimeWarning: invalid value encountered in true_divide
  return covariance_gufunc(x, y) / (x.std(axis=-1) * y.std(axis=-1))


<xarray.DataArray 'CC' (Parameters: 79)>
array([4.77503681e+01, 4.74619270e+01, 3.74114584e-01, 3.17223848e-01,
       2.83230076e-01, 2.64470207e-01, 2.46512389e-01, 2.35084073e-01,
       2.34867056e-01, 1.80091625e-01, 1.77853704e-01, 1.67287495e-01,
       1.47612404e-01, 1.35732782e-01, 1.13558242e-01, 1.03512161e-01,
       1.03487442e-01, 9.08711035e-02, 9.07483745e-02, 8.32280413e-02,
       8.21019920e-02, 8.19056249e-02, 8.17279398e-02, 8.02466728e-02,
       7.92531149e-02, 7.91888030e-02, 7.57120894e-02, 7.34691684e-02,
       7.08581066e-02, 6.60257418e-02, 6.11026312e-02, 5.80577462e-02,
       5.04215290e-02, 4.63602978e-02, 4.47123275e-02, 4.37520292e-02,
       4.34041969e-02, 4.21361528e-02, 2.99044303e-02, 2.99026501e-02,
       2.74785726e-02, 2.05329991e-02, 1.93428216e-02, 1.90900893e-02,
       1.55802337e-02, 1.53279939e-02, 1.47011788e-02, 1.19949846e-02,
       1.09632580e-02, 1.05122474e-02, 1.01670175e-02, 8.07980372e-03,
       6.34824612e-03, 6.18632437e-03, 6.13486702e-03, 5.28835171e-03,
       4.87079487e-03, 1.65124116e-03, 1.19451217e-03, 8.06849945e-04,
       5.56674994e-04, 3.55217793e-04, 2.02733687e-04, 1.94779928e-04,
       1.72021778e-04, 1.03905233e-04, 6.91452274e-05, 5.51361344e-05,
       1.40420564e-05, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00])
Coordinates:
  * Parameters  (Parameters) <U18 'pesticide_use' ... 'ironbirds'

# Export Results

Export MP, EP, LCI and CTV result to NetCDF file.

In [21]:
aeco.save(output_path, LCI=False)

c:\Users\giparoli\Documents\Projetos\AEco\Scripts\Tools_Iris.py:307: RuntimeWarning: invalid value encountered in true_divide
  return covariance_gufunc(x, y) / (x.std(axis=-1) * y.std(axis=-1))


LCIA saved at ./Outputs/Iris_outputs


In [22]:
#aeco.to_excel(output_path)